In [1]:
import pyscal.core as pc
import pyscal.crystal_structures as pcs
import os
import numpy as np
from tqdm import tqdm

In [2]:
from pymolsim import Sim, Thermostat, Barostat, Particle
from pymolsim.potential import LJ

ModuleNotFoundError: No module named 'dask'

In [3]:
sys = pc.System()
sys.read_inputfile("conf.dump", customkeys=["mass", "vx", "vy", "vz"])
atoms = sys.atoms
box = sys.box
particles = []
for atom in atoms:
    p = Particle()
    p.mass = 1.00;
    p.r = np.array(atom.pos);
    p.v = np.array([float(atom.custom["vx"]), float(atom.custom["vy"]), float(atom.custom["vz"])])
    p.type = atom.type
    particles.append(p)

In [4]:
sim = Sim(particles, box)

In [5]:
sim.beta = 0.5
sim.pressure = 0
sim.dt = 0.0025
sim.nthreads = 4

Create a potential

In [6]:
lj = LJ(1.0, 1.0)

Attch the potential to syatem

In [7]:
sim.potential = lj

We can start md now

In [8]:
sim.start()

In [9]:
sim.forces()

In [10]:
%%timeit
sim.forces()

20.7 s ± 428 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
sim.forces()

3.68 s ± 55.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
sim.forces()
sim.remap()

In [10]:
sim.dump(0)

In [11]:
for i in tqdm(range(1, 10)):
    sim.md_verlet()
    sim.remap()
    sim.dump(i)

100%|██████████| 9/9 [00:34<00:00,  3.84s/it]


In [14]:
from joblib import Parallel, delayed
import time

In [15]:
def addnum(i, j):
    time.sleep(0.01)
    return i+j

In [16]:
calcs = []
for i in range(10):
    for j in range(10):
        calcs.append(delayed (addnum) (i,j))

In [21]:
%%timeit
r = Parallel(n_jobs=4)(calcs)

274 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
100*0.01

1.0

In [11]:
from sklearn.metrics.pairwise import euclidean_distances

In [12]:
a =  np.array([np.array([1,1,1]), 
               np.array([2,2,2]), 
               np.array([3,3,3])])

In [13]:
euclidean_distances(a, a)

array([[0.        , 1.73205081, 3.46410162],
       [1.73205081, 0.        , 1.73205081],
       [3.46410162, 1.73205081, 0.        ]])

In [52]:
x = np.array([0.0, 0.5, 0.0, 0.0])
y = np.array([0.0, 0.0, 0.5, 0.0])
z = np.array([0.0, 0.0, 0.0, 0.5])
o = np.array([0.0, 1.0, 0.0, 0.0])

In [63]:
xx = np.zeros((len(x), len(x)))
for i in range(len(x)):
    for j in range(len(x)):
        xx[i, j] = x[i] - x[j]

In [74]:
xx

array([[ 0. , -0.5,  0. ,  0. ],
       [ 0.5,  0. ,  0.5,  0.5],
       [ 0. , -0.5,  0. ,  0. ],
       [ 0. , -0.5,  0. ,  0. ]])

In [79]:
xd = np.meshgrid(x, x)[1] - np.meshgrid(x, x)[0]
yd = np.meshgrid(y, y)[1] - np.meshgrid(y, y)[0]
zd = np.meshgrid(z, z)[1] - np.meshgrid(z, z)[0]

In [82]:
(xd**2 + yd**2 + zd**2)**0.5

array([[0.        , 0.5       , 0.5       , 0.5       ],
       [0.5       , 0.        , 0.70710678, 0.70710678],
       [0.5       , 0.70710678, 0.        , 0.70710678],
       [0.5       , 0.70710678, 0.70710678, 0.        ]])

In [83]:
def fsum(x, y, z):
    return x+y+z

In [84]:
fsum(xd, yd, zd)

array([[ 0. , -0.5, -0.5, -0.5],
       [ 0.5,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ,  0. ]])

In [87]:
xd

array([[ 0. , -0.5,  0. ,  0. ],
       [ 0.5,  0. ,  0.5,  0.5],
       [ 0. , -0.5,  0. ,  0. ],
       [ 0. , -0.5,  0. ,  0. ]])

In [85]:
yd

array([[ 0. ,  0. , -0.5,  0. ],
       [ 0. ,  0. , -0.5,  0. ],
       [ 0.5,  0.5,  0. ,  0.5],
       [ 0. ,  0. , -0.5,  0. ]])

In [86]:
zd

array([[ 0. ,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. , -0.5],
       [ 0.5,  0.5,  0.5,  0. ]])